In [14]:
import numpy as np
import tensorflow as tf
from tf_agents.networks.q_network import QNetwork
import tf_agents.networks.network as network
from tf_agents.specs import tensor_spec
from snake_game import SnakeGame
from scene import Scene
from scene_longer_snake import SceneLongerSnake
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input

scene = SceneLongerSnake(init_randomly=True, snake_longer_prob=0.95, length_mean=10, length_std=2)
episodes_count = 100000
random_episodes = 20000


In [15]:
from tf_agents.environments.tf_py_environment import TFPyEnvironment

env = SnakeGame(scene)
env = TFPyEnvironment(env)


In [16]:
class MyQNetwork(network.Network):
    def __init__(self,
                 input_tensor_spec,
                 action_spec,
                 name='MyQNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=input_tensor_spec,
            state_spec=(),
            name=name)
        
        self._action_spec = action_spec
        matrix_shape = input_tensor_spec[0].shape
        obstacles_shape = input_tensor_spec[1].shape
        no_body_blocks_shape = input_tensor_spec[2].shape
        
        input0 = Input(shape=matrix_shape)
        input1 = Input(shape=obstacles_shape) # Each value represents if there is a wall or the snake body in the direction, 0 - no wall, 1 - wall
        input2 = Input(shape=no_body_blocks_shape) # Each value represents how many no body blocks are in the direction
        
        conv = Conv2D(32, (2, 2), 1, activation='relu', kernel_initializer='he_normal')(input0)
        flat = Flatten()(conv)
        dense1 = Dense(128, activation='relu', kernel_initializer='he_normal')(flat)
        dense2 = Dense(32, activation='relu', kernel_initializer='he_normal')(dense1)
        
        concat = tf.keras.layers.concatenate([dense2, input1, input2])
        output = Dense(4, activation='linear')(concat)
        
        self._model = tf.keras.Model(inputs=[input0, input1, input2], outputs=output)
        
        self._model.summary()

    def call(self, observations, step_type=None, network_state=(), training=False):
        output = self._model([observations[0], observations[1], observations[2]])
        return output, network_state

q_net = MyQNetwork(
    env.observation_spec(),
    env.action_spec())


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d_2 (Conv2D)           (None, 4, 4, 32)             544       ['input_7[0][0]']             
                                                                                                  
 flatten_2 (Flatten)         (None, 512)                  0         ['conv2d_2[0][0]']            
                                                                                                  
 dense_6 (Dense)             (None, 128)                  65664     ['flatten_2[0][0]']           
                                                                                            

In [17]:
# Create the agent
from tf_agents.agents.dqn.dqn_agent import DdqnAgent
from tensorflow import keras
from keras.optimizers.legacy import Adam
from keras.losses import Huber
from tf_agents.trajectories import TimeStep

train_step_counter = tf.Variable(0)
optimizer = Adam(learning_rate=0.001)

epsilon = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.1,
    decay_steps=episodes_count,
    end_learning_rate=0.01
)

agent = DdqnAgent(
    time_step_spec=env.time_step_spec(),
    action_spec=env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=Huber(reduction="none"),
    gamma=tf.constant(0.95, dtype=tf.float32),
    train_step_counter=train_step_counter,
    epsilon_greedy=lambda: epsilon(train_step_counter)
)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 5, 5, 4)]            0         []                            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 4, 4, 32)             544       ['input_10[0][0]']            
                                                                                                  
 flatten_3 (Flatten)         (None, 512)                  0         ['conv2d_3[0][0]']            
                                                                                                  
 dense_9 (Dense)             (None, 128)                  65664     ['flatten_3[0][0]']           
                                                                                            

In [18]:
# Create the replay buffer
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=100000
)


In [19]:
# Create the training metrics
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric()
]


In [20]:
# Create the driver
from tf_agents.drivers import dynamic_step_driver

collect_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=agent.collect_policy,
    observers=[replay_buffer.add_batch] + train_metrics,
    num_steps=2
)

# Run a random policy to fill the replay buffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(env.time_step_spec(), env.action_spec())
init_driver = dynamic_step_driver.DynamicStepDriver(
    env,
    policy=initial_collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=random_episodes
)

final_time_step, final_policy_state = init_driver.run()


In [21]:
# Create the dataset
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2, # Capaz cambiar esto
    num_parallel_calls=5
).prefetch(5)


In [22]:

# Create the training loop

from tf_agents.utils.common import function
from tf_agents.trajectories import trajectory
from tf_agents.eval.metric_utils import log_metrics
import logging

logging.getLogger().setLevel(logging.INFO)

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

def train_agent(n_iterations):
    time_step = None
    policy_state = agent.collect_policy.get_initial_state(env.batch_size)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        if iteration % 1000 == 0:
            print("Iteration: ", iteration)
            print("Replay buffer len: " + str(replay_buffer.num_frames().numpy()))
            log_metrics(train_metrics)


In [23]:
train_agent(episodes_count)


INFO:absl: 
		 NumberOfEpisodes = 2
		 EnvironmentSteps = 2
		 AverageReturn = -60.0
		 AverageEpisodeLength = 0.5


Iteration:  0
Replay buffer len: 43702


INFO:absl: 
		 NumberOfEpisodes = 672
		 EnvironmentSteps = 2002
		 AverageReturn = -26223.0
		 AverageEpisodeLength = 5.199999809265137


Iteration:  1000
Replay buffer len: 46372


INFO:absl: 
		 NumberOfEpisodes = 914
		 EnvironmentSteps = 4002
		 AverageReturn = -35406.0
		 AverageEpisodeLength = 3.700000047683716


...

Iteration:  97000
Replay buffer len: 100000


INFO:absl: 
		 NumberOfEpisodes = 9718
		 EnvironmentSteps = 196002
		 AverageReturn = -228432.0
		 AverageEpisodeLength = 39.20000076293945


Iteration:  98000
Replay buffer len: 100000


INFO:absl: 
		 NumberOfEpisodes = 9770
		 EnvironmentSteps = 198002
		 AverageReturn = -228383.0
		 AverageEpisodeLength = 41.099998474121094


Iteration:  99000
Replay buffer len: 100000


In [24]:
# Evaluate the agent

test_scene = Scene(init_randomly=True)
test_env = SnakeGame(test_scene)
test_env = TFPyEnvironment(test_env)

time_step = test_env.reset()
rewards = []
steps = 0

while not time_step.is_last() and steps < 200:
    steps += 1
    print("Map:\n", np.argmax(time_step.observation[0], axis=3))
    print("Direction:\n", time_step.observation[1])
    print("No body blocks:\n", time_step.observation[2])
    action_step = agent.policy.action(time_step)
    print("Action: ", action_step.action.numpy())
    time_step = test_env.step(action_step.action)
    rewards.append(time_step.reward.numpy()[0])
    
print("Total reward: ", sum(rewards))
print("Total steps: ", steps)


Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 3 0 0 0]
  [0 0 0 0 0]
  [1 2 0 0 0]]]
Direction:
 tf.Tensor([[ True False  True  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[0.  0.5 0.5 0. ]], shape=(1, 4), dtype=float32)
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 3 0 0 0]
  [1 0 0 0 0]
  [2 0 0 0 0]]]
Direction:
 tf.Tensor([[ True False False  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[0.    0.375 0.5   0.125]], shape=(1, 4), dtype=float32)
Action:  [2]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 3 0 0 0]
  [2 1 0 0 0]
  [0 0 0 0 0]]]
Direction:
 tf.Tensor([[ True False False False]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[0.125 0.375 0.375 0.125]], shape=(1, 4), dtype=float32)
Action:  [1]
Map:
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 1 0 0 0]
  [2 2 0 0 0]
  [0 0 3 0 0]]]
Direction:
 tf.Tensor([[False False False  True]], shape=(1, 4), dtype=bool)
No body blocks:
 tf.Tensor([[0.12820514 0.25641027 0.3846154  0.23076923]], shape=(1, 4), dtyp

In [25]:

# Count the average steps per episode

n = 500
total_steps = 0
total_reward = 0
steps = 0
total_500 = 0
wins = 0

for i in range(n):
    if i % 10 == 0 and i != 0:
        print("Episode: ", i)
        
    time_step = test_env.reset()
    steps = 0
    episode_reward = 0
    while not time_step.is_last() and steps < 500:
        steps += 1
        action_step = agent.policy.action(time_step)
        time_step = test_env.step(action_step.action)
        reward = time_step.reward.numpy()[0]
        total_reward += reward
        episode_reward += reward
        
    if episode_reward == 115:
        wins += 1
    
    if steps == 500:
        total_500 += 1
    else:
        total_steps += steps
        
    if i % 10 == 0 and i != 0:
        print("Avg reward: ", total_reward / (i + 1))
        
if steps == 500:
    print("500 episodes reached")
    
print("Average steps per episode: ", total_steps / (n - total_500))
print("Average reward per episode: ", total_reward / n)
print("Total 500 episodes: ", total_500)
print("Wins: ", wins)


Episode:  10
Avg reward:  44.09090909090909
Episode:  20
Avg reward:  38.095238095238095
Episode:  30
Avg reward:  39.03225806451613
Episode:  40
Avg reward:  42.19512195121951
Episode:  50
Avg reward:  41.76470588235294
Episode:  60
Avg reward:  41.721311475409834
Episode:  70
Avg reward:  43.66197183098591
Episode:  80
Avg reward:  44.44444444444444
Episode:  90
Avg reward:  44.010989010989015
Episode:  100
Avg reward:  44.20792079207921
Episode:  110
Avg reward:  45.04504504504504
Episode:  120
Avg reward:  44.79338842975206
Episode:  130
Avg reward:  45.725190839694655
Episode:  140
Avg reward:  44.57446808510638
Episode:  150
Avg reward:  45.2317880794702
Episode:  160
Avg reward:  45.43478260869565
Episode:  170
Avg reward:  45.84795321637427
Episode:  180
Avg reward:  46.43646408839779
Episode:  190
Avg reward:  46.49214659685864
Episode:  200
Avg reward:  47.08955223880597
Episode:  210
Avg reward:  47.03791469194313
Episode:  220
Avg reward:  47.23981900452489
Episode:  230
Av

In [26]:
# Save the q network weights
tf.keras.models.save_model(q_net._model, "q_network_x.h5")

/var/folders/bw/_7s8wxw93cngpt5f7bg5s3hm0000gn/T/ipykernel_28307/4087533747.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(q_net._model, "q_network_5x5_3.h5")
